In [1]:
import torch
from torch import nn
import numpy as np
import pandas as pd
import PIL
import glob
import os
import torchvision
import gym
from pyvirtualdisplay import Display
import matplotlib.pyplot as plt
import copy
import time
import random

In [2]:
DQN=torch.nn.Sequential(nn.Linear(in_features=2,out_features=32,bias=True),
                       nn.ReLU(),
                       nn.Linear(in_features=32,out_features=64,bias=True),
                       nn.ReLU(),
                       nn.Linear(in_features=64,out_features=128,bias=True),
                       nn.ReLU(),
                       nn.Linear(in_features=128,out_features=256,bias=True),
                       nn.ReLU(),
                       nn.Linear(in_features=256,out_features=512,bias=True),
                       nn.ReLU(),
                       nn.Linear(in_features=512,out_features=256,bias=True),
                       nn.ReLU(),
                       nn.Linear(in_features=256,out_features=3,bias=True))
targetDQN=copy.deepcopy(DQN)
for i,j in zip(DQN.parameters(),targetDQN.parameters()):
    (j.data).copy_(i.data)
for i in targetDQN.parameters():
    i.requires_grad=False

In [10]:
opt=torch.optim.Adam(DQN.parameters(),lr=0.0005)

In [14]:
#使用的算法：TD（Q-learning）、Double DQN、Multiple Step、Experience Replay
%matplotlib qt5
buffer_lengthest=50000
buffer_length_to_train=10000
minibatch_size=50
discount=0.98
env=gym.make('MountainCar-v0',render_mode='rgb_array')
observation,info=env.reset()
#plt.figure('CartPole-v1',figsize=(4,4))
update_target_DQN=0
alpha_greedy=0.00001
steps=0
# replay_buffer=[]
while(1):
    small_buffer=[]#创建小buffer，再整体存入replay_buffer中
    #plt.clf()#清除当前图像，必须要有，否则图片会积压住，后来很影响整体代码运行速度
    small_buffer.append(list(observation))
    #plt.imshow(env.render())#显示图像
    #plt.pause(0.00000001)#暂停一小会，必须有
    a=random.random()#使用α-greedy算法，先取一个随机数
    act=DQN(torch.from_numpy(observation))#DQN运算
    if a>alpha_greedy:#如果随机数大于阈值，选DQN分值最大的
        action=torch.argmax(act).item()
        #scoret=torch.max(act)#获得当前t时刻预测的分数
    else:#否则，随机选择动作
        action=random.randint(0,2)
        #scoret=act[action]#获得当前t时刻预测的分数
    small_buffer.append(action)
    observation, reward, terminated, truncated, info=env.step(action)#环境执行动作，获得新观测值、奖励以及其他
    if observation[0]>-0.4 and observation[0]<0.5:
        reward=10*(observation[0]+0.4)**3
    elif observation[0]>=0.5:
        reward=100
    elif observation[0]<=-0.4:
        reward=-0.8
    small_buffer.append(reward)
    small_buffer.append(list(observation))
    print('action:',action,'reward:',reward)
    steps+=1
    #下面是开始训练的代码：
    if len(replay_buffer)>=buffer_length_to_train:#当replay_buffer中的数据长度超过buffer_length_to_train的时候才开始训练
        if len(replay_buffer)==buffer_length_to_train:
            print('Start Train')
        how_many=0#在本代码算法结构中，如果在multi_step中遇到自然结束（500步），就舍弃掉这个，how_many就是来计算实际有多少个梯度，用来计算梯度平均
        opt.zero_grad()#将DQN梯度清零
        for epoch in range(minibatch_size):#建立循环，minibatch_size个
            drop_this_epoch=0#标识符，用来判断
            end_by_fail=0#标识符，用来判断
            target_score=0#TD算法的目标分数，初始化为0
            start_num=random.randint(0,len(replay_buffer)-51)#要在replay_buffer中取样本，用随机数生成第一个数的序号
            len_mult_step=random.randint(2,50)#训练使用multipe_step,也用随机数生成step距离
            for i in range(len_mult_step):
                if replay_buffer[start_num+i][4]==1:#杆子倒了，失败了
                    len_mult_step=i+1
                    end_by_fail=1#标识符
                    break
                elif replay_buffer[start_num+i][4]==2:#超过500轮，自动停止
                    drop_this_epoch=1#标识符，意味舍弃这一组数据，不计算梯度
                    break
            if drop_this_epoch==1:
                continue#如果遇到自然停止的（超过500），跳出这一轮循环，不计算梯度
            scoret=(DQN(torch.Tensor(replay_buffer[start_num][0])))[replay_buffer[start_num][1]]#获得t时刻分数
            if end_by_fail==1:#如果遇到失败的清空，target_score就是折扣汇报的累加，不需要targetDQN计算
                for i in range(len_mult_step):
                    target_score+=replay_buffer[start_num+i][2]*pow(discount,i)
            else:
                for i in range(len_mult_step):
                    target_score+=replay_buffer[start_num+i][2]*pow(discount,i)
                target_score+=torch.max(targetDQN(torch.Tensor(replay_buffer[start_num+len_mult_step-1][3])))*pow(discount,len_mult_step)
            loss=0.5*pow((scoret-target_score),2)
            loss.backward()
            how_many+=1
        for i in DQN.parameters():#loss.backward会将梯度累加，这里求梯度的平均
            (i.grad.data).div_(how_many)
#             print(i.grad)
        opt.step()#优化器执行
        #print('optim step successfully')
        update_target_DQN+=1
        if update_target_DQN==30:#更新targetDQN
            update_target_DQN=0
            for i,j in zip(DQN.parameters(),targetDQN.parameters()):
                j.data.mul_(0.3)
                j.data.add_((i.data)*0.7)
    ################################################
    if not (truncated or terminated):
        small_buffer.append(0)
        if len(replay_buffer)>=buffer_lengthest:
            del replay_buffer[0]#如果replay_buffer的总长度大于阈值（3000），则添加新的时候，删掉最老的（序号为0的）
        replay_buffer.append(small_buffer)
    elif terminated:#成功到达终点
        small_buffer.append(1)
        small_buffer[2]+=5000
        if len(replay_buffer)>=buffer_lengthest:
            del replay_buffer[0]
        replay_buffer.append(small_buffer)
        print('Episode End , steps=', steps)
        steps=0
        print(len(replay_buffer))
        observation,info=env.reset()
    elif truncated:#超过200轮，自动停止
        small_buffer.append(2)
        if len(replay_buffer)>=buffer_lengthest:
            del replay_buffer[0]
        replay_buffer.append(small_buffer)
        print('Episode End by 200')
        steps=0
        observation,info=env.reset()

action: 0 reward: -0.8
action: 0 reward: -0.8
action: 0 reward: -0.8
action: 0 reward: -0.8
action: 0 reward: -0.8
action: 0 reward: -0.8
action: 0 reward: -0.8
action: 0 reward: -0.8
action: 0 reward: -0.8
action: 0 reward: -0.8
action: 0 reward: -0.8
action: 0 reward: -0.8
action: 0 reward: -0.8
action: 0 reward: -0.8
action: 0 reward: -0.8
action: 0 reward: -0.8
action: 0 reward: -0.8
action: 0 reward: -0.8
action: 0 reward: -0.8
action: 0 reward: -0.8
action: 0 reward: -0.8
action: 0 reward: -0.8
action: 0 reward: -0.8
action: 0 reward: -0.8
action: 0 reward: -0.8
action: 0 reward: -0.8
action: 0 reward: -0.8
action: 0 reward: -0.8
action: 0 reward: -0.8
action: 0 reward: -0.8
action: 2 reward: -0.8
action: 2 reward: -0.8
action: 2 reward: -0.8
action: 2 reward: -0.8
action: 2 reward: -0.8
action: 2 reward: -0.8
action: 2 reward: -0.8
action: 2 reward: -0.8
action: 2 reward: -0.8
action: 2 reward: -0.8
action: 2 reward: -0.8
action: 2 reward: -0.8
action: 2 reward: -0.8
action: 2 r

action: 0 reward: -0.8
action: 0 reward: -0.8
action: 0 reward: -0.8
action: 0 reward: -0.8
action: 0 reward: -0.8
action: 0 reward: -0.8
action: 0 reward: -0.8
action: 0 reward: -0.8
action: 0 reward: -0.8
action: 0 reward: -0.8
action: 0 reward: -0.8
action: 0 reward: -0.8
action: 0 reward: -0.8
action: 0 reward: -0.8
action: 0 reward: -0.8
action: 0 reward: -0.8
action: 0 reward: -0.8
action: 2 reward: -0.8
action: 2 reward: -0.8
action: 2 reward: -0.8
action: 2 reward: -0.8
action: 2 reward: -0.8
action: 2 reward: -0.8
action: 2 reward: -0.8
action: 2 reward: -0.8
action: 2 reward: -0.8
action: 2 reward: -0.8
action: 2 reward: -0.8
action: 2 reward: -0.8
action: 2 reward: -0.8
action: 2 reward: -0.8
action: 2 reward: -0.8
action: 2 reward: -0.8
action: 2 reward: -0.8
action: 2 reward: -0.8
action: 2 reward: -0.8
action: 2 reward: -0.8
action: 2 reward: -0.8
action: 2 reward: -0.8
action: 2 reward: -0.8
action: 2 reward: 0.0006955492949953985
action: 2 reward: 0.005968872357173618
a

action: 2 reward: -0.8
action: 2 reward: -0.8
action: 0 reward: -0.8
action: 0 reward: -0.8
action: 0 reward: -0.8
action: 0 reward: -0.8
action: 0 reward: -0.8
action: 0 reward: -0.8
action: 0 reward: -0.8
action: 0 reward: -0.8
action: 0 reward: -0.8
action: 0 reward: -0.8
action: 0 reward: -0.8
action: 0 reward: -0.8
action: 0 reward: -0.8
action: 0 reward: -0.8
action: 0 reward: -0.8
action: 0 reward: -0.8
action: 0 reward: -0.8
action: 0 reward: -0.8
action: 0 reward: -0.8
action: 0 reward: -0.8
action: 0 reward: -0.8
action: 0 reward: -0.8
action: 0 reward: -0.8
action: 0 reward: -0.8
action: 0 reward: -0.8
action: 0 reward: -0.8
action: 0 reward: -0.8
action: 0 reward: -0.8
action: 0 reward: -0.8
action: 0 reward: -0.8
action: 0 reward: -0.8
action: 0 reward: -0.8
action: 0 reward: -0.8
action: 0 reward: -0.8
action: 0 reward: -0.8
action: 0 reward: -0.8
action: 0 reward: -0.8
action: 2 reward: -0.8
action: 2 reward: -0.8
action: 2 reward: -0.8
action: 2 reward: -0.8
action: 2 r

action: 2 reward: 6.475317074412345
action: 2 reward: 6.839032706433778
action: 2 reward: 7.232585019143473
action: 2 reward: 100
Episode End , steps= 96
10925
action: 0 reward: -0.8
action: 0 reward: -0.8
action: 0 reward: -0.8
action: 0 reward: -0.8
action: 0 reward: -0.8
action: 0 reward: -0.8
action: 0 reward: -0.8
action: 0 reward: -0.8
action: 0 reward: -0.8
action: 0 reward: -0.8
action: 0 reward: -0.8
action: 0 reward: -0.8
action: 0 reward: -0.8
action: 0 reward: -0.8
action: 0 reward: -0.8
action: 0 reward: -0.8
action: 0 reward: -0.8
action: 0 reward: -0.8
action: 0 reward: -0.8
action: 0 reward: -0.8
action: 0 reward: -0.8
action: 0 reward: -0.8
action: 0 reward: -0.8
action: 0 reward: -0.8
action: 0 reward: -0.8
action: 0 reward: -0.8
action: 0 reward: -0.8
action: 0 reward: -0.8
action: 0 reward: -0.8
action: 0 reward: -0.8
action: 2 reward: -0.8
action: 2 reward: -0.8
action: 2 reward: -0.8
action: 2 reward: -0.8
action: 2 reward: -0.8
action: 2 reward: -0.8
action: 2 re

action: 2 reward: 2.6573621788628996
action: 2 reward: 2.98877782558026
action: 2 reward: 3.3360983241709317
action: 2 reward: 3.7001800310113726
action: 2 reward: 4.082454160637965
action: 2 reward: 4.484973780798422
action: 2 reward: 4.910464625863059
action: 2 reward: 5.362391944416752
action: 2 reward: 5.845037631513389
action: 2 reward: 6.3636030996562045
action: 2 reward: 6.924332132595046
action: 2 reward: 100
Episode End , steps= 104
11222
action: 0 reward: -0.8
action: 0 reward: -0.8
action: 0 reward: -0.8
action: 0 reward: -0.8
action: 0 reward: -0.8
action: 0 reward: -0.8
action: 0 reward: -0.8
action: 0 reward: -0.8
action: 0 reward: -0.8
action: 0 reward: -0.8
action: 0 reward: -0.8
action: 0 reward: -0.8
action: 0 reward: -0.8
action: 0 reward: -0.8
action: 0 reward: -0.8
action: 0 reward: -0.8
action: 0 reward: -0.8
action: 0 reward: -0.8
action: 0 reward: -0.8
action: 0 reward: -0.8
action: 0 reward: -0.8
action: 0 reward: -0.8
action: 0 reward: -0.8
action: 0 reward: -

KeyboardInterrupt: 

In [18]:
%matplotlib qt5
env=gym.make('MountainCar-v0',render_mode='rgb_array')
observation, info=env.reset()
plt.figure('MountainCar-v0',figsize=(4,4))
steps=0
for t in range(1000):
    plt.clf()
    plt.imshow(env.render())
    plt.pause(0.00001)
    #print(observation)
    with torch.no_grad():
        action=torch.argmax(DQN(torch.from_numpy(observation))).item()
    #print(aa.type())
    #b=DQN(aa)
    #print(b)
    #print(torch.from_numpy(observation))
    #action=env.action_space.sample()
    #print(action)
    observation, reward, terminated, truncated, info=env.step(action)
    steps+=1
    print('action',action)
    #print(observation.type())
    if truncated:
        print('Finished by 200')
        break
    elif terminated:
        print('Finished, steps=',steps)
        break
env.close()

action 2
action 2
action 2
action 2
action 2
action 2
action 2
action 2
action 2
action 2
action 2
action 2
action 2
action 2
action 2
action 2
action 2
action 0
action 0
action 0
action 0
action 0
action 0
action 0
action 0
action 0
action 0
action 0
action 0
action 0
action 0
action 0
action 0
action 0
action 0
action 0
action 0
action 0
action 0
action 0
action 0
action 0
action 0
action 0
action 0
action 0
action 0
action 0
action 0
action 2
action 2
action 2
action 2
action 2
action 2
action 2
action 2
action 2
action 2
action 2
action 2
action 2
action 2
action 2
action 2
action 2
action 2
action 2
action 2
action 2
action 2
action 2
action 2
action 2
action 2
action 2
action 2
action 2
action 2
action 2
action 2
action 2
action 2
action 2
action 2
action 2
action 2
action 2
action 2
action 2
action 2
action 2
action 2
action 2
action 2
action 2
action 2
action 2
action 2
action 2
action 2
action 2
action 2
Finished, steps= 103


In [20]:
torch.save(DQN,'./DDQN_MountainCar_full.pth')